In [1]:
import json
from confluent_kafka import Consumer, KafkaError
from datetime import datetime, timedelta
import boto3
import os
import shutil

In [2]:
#test code
current_directory = os.getcwd()
counter = 0
access_key = ''
secret_access_key = ''
#region_name='US East (N. Virginia)'
region_name='us-east-1'

s3 = boto3.client('s3',
                  aws_access_key_id = access_key,
                  aws_secret_access_key = secret_access_key,
                  region_name = region_name)

bucket = 'awsdwdiproswsp'

# # Walk through the directory
# for root, dirs, files in os.walk(current_directory):
#     for file in files:
        
#         if file.endswith('.json'):
#             file_path = os.path.join(root, file)
#             # Upload file to S3
#             #s3.upload_file(os.path.join(current_directory, file),bucket, 'staging_json_dwdiproj_swsp/'+file)
#             counter += 1
#             shutil.move(os.path.join(current_directory, file), os.path.join(current_directory,'Processed_JSONs'))


In [3]:
# Kafka broker address

#bootstrap_servers = '155.33.135.45:9092'
bootstrap_servers = 'localhost:9092'

# Kafka topic to consume messages from
topic = 'JSON_ingestion'

# Create Kafka Consumer instance
consumer = Consumer({
    'bootstrap.servers': bootstrap_servers,
    'group.id': 'my_consumer_group',
    'auto.offset.reset': 'earliest'  # Start consuming from the beginning of the topic
})

# Subscribe to the topic
consumer.subscribe([topic])

In [4]:
# Buffer to store messages before writing to file
message_buffer = b""
max_buffer_size_bytes = 10240 * 1024  # Maximum buffer size in bytes (1MB in this case)
current_directory = os.getcwd()

try:
    while True:
        counter = 0
        msg = consumer.poll(timeout=1.0)
        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition
                print(f"Reached end of partition {msg.partition()}")
            elif msg.error():
                # Error
                print(f"Error: {msg.error()}")
        else:
            # Message
            message_bytes = msg.value()
            print(f"Received message partition: {msg.partition()} offset: {msg.offset()}")

            # Check if appending this message will exceed the buffer size
            if len(message_buffer) + len(message_bytes) >= max_buffer_size_bytes:
                # Write buffered messages to file
                timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
                file_name = f"WebData_{timestamp}.json"
                with open(file_name, 'wb') as file:
                    file.write(message_buffer)
                    print(f'Writing to file {file_name}')
                # Clear the buffer after writing to file
                
                #s3 = boto3.client('s3')
                
                
                # Walk through the directory
                
                for file in os.listdir(current_directory):
                    #print("Processing file: " + file)
                    if counter >= 5:
                        continue
                    if file.endswith('.json') and file.startswith('WebData'):
                        file_path = os.path.join(current_directory, file)
                        if os.path.exists(file_path):  # Check if the file exists
                            # Upload file to S3
                            s3.upload_file(file_path, bucket, 'staging_json_dwdiproj_swsp/' + file)
                            counter += 1
                            # Move file to Processed_JSONs directory
                            if not os.path.exists(os.path.join(os.path.join(current_directory,'Processed_JSONs'),file)):
                                shutil.move(file_path, os.path.join(current_directory, 'Processed_JSONs'))
                        else:
                            print(f"File '{file_path}' does not exist.")

                message_buffer = b""

            # Append the message to the buffer
            message_buffer += message_bytes

except KeyboardInterrupt:
    # If there are any remaining messages in the buffer, write them to file
    if message_buffer:
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        file_name = f"WebData_{timestamp}_3.json"
        with open(file_name, 'wb') as file:
            file.write(message_buffer)
        
    # Close the consumer
    consumer.close()

finally:
    # If there are any remaining messages in the buffer, write them to file
    if message_buffer:
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        file_name = f"WebData_{timestamp}_2.json"
        with open(file_name, 'wb') as file:
            file.write(message_buffer)
    
    # Close the consumer
    consumer.close()

Received message partition: 0 offset: 9038
Received message partition: 0 offset: 9039
Received message partition: 0 offset: 9040
Received message partition: 0 offset: 9041
Received message partition: 0 offset: 9042
Received message partition: 0 offset: 9043
Received message partition: 0 offset: 9044
Received message partition: 0 offset: 9045
Received message partition: 0 offset: 9046
Received message partition: 0 offset: 9047
Received message partition: 0 offset: 9048
Received message partition: 0 offset: 9049
Received message partition: 0 offset: 9050
Received message partition: 0 offset: 9051
Received message partition: 0 offset: 9052
Received message partition: 0 offset: 9053
Received message partition: 0 offset: 9054
Received message partition: 0 offset: 9055
Received message partition: 0 offset: 9056
Received message partition: 0 offset: 9057
Received message partition: 0 offset: 9058
Received message partition: 0 offset: 9059
Received message partition: 0 offset: 9060
Received me